# 时间序列平稳性检验

判断时间序列数据是否为平稳时间序列或非平稳时间序列，通常可以通过以下方法：

（1）观察时间序列数据的均值和方差是否随时间变化而发生明显的改变。若均值和方差变化明显，则该时间序列数据可能为非平稳时间序列，反之，则可能为平稳时间序列。

（2）对时间序列数据进行差分后，再对数据通过ADF单位根检验或KPSS检验，如果数据平稳，则该时间序列数据可能为非平稳时间序列，反之，则可能为平稳时间序列。

In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics import euclidean_distances
from scipy.spatial.distance import cdist
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签SimHei
plt.rcParams['axes.unicode_minus'] = False # 用来正常显示负号
import warnings #消除警告
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter('ignore', InterpolationWarning)

In [21]:
#选取的数据为正常工况的数据
data = pd.read_csv("../data/DATA_GT1_CMS_201508-201604.csv", encoding="gbk")
data.to_pickle("../data/DATA_GT1_CMS_201508-201604.pkl")
data.head(5)

,Time,DWATT,CSGV,CTIM,TTXD1_1,TTXD1_2,TTXD1_3,TTXD1_4,TTXD1_5,TTXD1_6,...,TTXD1_19,TTXD1_20,TTXD1_21,TTXD1_22,TTXD1_23,TTXD1_24,TTXM,TTXSP1,TTXSP3,TTXSP2
0,#!Units,MW,na,deg F,deg F,deg F,deg F,deg F,deg F,deg F,...,deg F,deg F,deg F,naF,naF,naF,deg F,deg F,deg F,deg F
1,#!Description,Generator Watts Max Selected,IGV angle in deg,Compressor Inlet Temperature,Exhaust Thermocouple 1- Compensated,Exhaust Thermocouple 2- Compensated,Exhaust Thermocouple 3- Compensated,Exhaust Thermocouple 4- Compensated,Exhaust Thermocouple 5- Compensated,Exhaust Thermocouple 6 - Compensated,...,Exhaust Thermocouple 19 - Compensated,Exhaust Thermocouple 20 - Compensated,Exhaust Thermocouple 21 - Compensated,Exhaust Thermocouple 22 - Compensated,Exhaust Thermocouple 23 - Compensated,Exhaust Thermocouple 24 - Compensated,Exhaust Temp Median Corrected By Average,Combustion Monitor Actual Spread 1,Combustion Monitor Actual Spread 3,Combustion Monitor Actual Spread 2
2,2015/8/1 0:00,103.0826721,84.05797577,91.35520935,1056.271606,1056.249512,1056.90564,1039.57605,1051.324463,1071.620239,...,1040.360107,1044.769287,1061.994507,1052.054688,1055.862671,1061.955322,1055.949707,25.66916656,23.58292007,24.93045998
3,2015/8/1 1:00,103.3062134,84.06842041,89.63658905,1057.366943,1055.130981,1057.074707,1037.682373,1049.881348,1070.99231,...,1040.169312,1042.369751,1060.376343,1050.632324,1053.875122,1060.029053,1052.199707,25.02660942,22.51792336,23.52773857
4,2015/8/1 2:00,103.5618591,84.05568695,89.13874054,1055.688599,1053.412842,1056.954224,1039.199585,1050.531128,1071.470703,...,1040.676514,1041.763428,1062.228394,1050.305054,1056.360107,1059.859985,1054.869141,26.2706337,22.97101974,22.13184357


In [22]:
#剔除自变量
filter_data = data.drop("Time", axis=1).iloc[3:-1, :].reset_index(drop=True).astype('float32')
filter_data.shape

(3300, 31)

# 差分

为什么先差分再检验平稳性？

- 大部分经典时间序列模型，如ARIMA模型，要求时间序列是平稳的。只有当时间序列平稳时，才能有效运用这些模型来进行预测和建模。
- 差分操作可以去除时间序列中的趋势、季节性等非平稳性因素，使得原始数据更符合平稳性的要求。
- 差分操作可以消除时间序列的自相关性，使得之后的模型拟合和预测更加准确可靠。

In [23]:
# data为时间序列数据，n为差分次数，默认为1
def difference(data, n=1):
    diff = data.diff(n)
    diff.dropna(inplace=True)
    return diff

# 差分1次
diff_data = difference(filter_data)

In [24]:
diff_data.shape

(3299, 31)

# ADF单位根检验

ADF单位根检验：检验结果包括ADF统计量、p值以及临界值。若 p 值小于显著性水平（常见使用的是0.05），则可以拒绝原假设，即数据是平稳的，反之。

In [32]:
from statsmodels.tsa.stattools import adfuller

# data为时间序列数据
result = adfuller(diff_data.iloc[:, 3].values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))
    
if result[1]<0.05:
    print("该序列是平稳的")
else:
    print("该序列是非平稳的")

ADF Statistic: -20.442174
p-value: 0.000000
Critical Values:
	1%: -3.432
	5%: -2.862
	10%: -2.567
该序列是平稳的


# KPSS检验

KPSS检验：检验结果包括KPSS统计量、p值以及临界值。若 p 值小于显著性水平，则可以拒绝原假设，即数据是非平稳的，反之。

In [31]:
from statsmodels.tsa.stattools import kpss

# data为时间序列数据
result = kpss(diff_data.iloc[:, 0].values)
print('KPSS Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[3].items():
    print('\t%s: %.3f' % (key, value))
    
if result[1]>0.05:
    print("该序列是平稳的")
else:
    print("该序列是非平稳的")

KPSS Statistic: 0.041736
p-value: 0.100000
Critical Values:
	10%: 0.347
	5%: 0.463
	2.5%: 0.574
	1%: 0.739
该序列是平稳的。
